In [1]:
import urllib.request, json , time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime

In [2]:
try:
    import httplib
except:
    import http.client as httplib

def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        print("True")
        return True
    except:
        conn.close()
        print("False")
        return False

In [3]:
def get_historic_price(query_url,json_path,csv_path):
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)
    
    stock_id=query_url.split("&period")[0].split("symbol=")[1]
    
    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists, Updating data...")

        try:
            with urllib.request.urlopen(query_url) as url:
                parsed = json.loads(url.read().decode())
        except:
            print("|||  Historical data of "+stock_id+" doesn't exist")
            return
    
    else:
        if os.path.exists(json_path+stock_id+'.json'):
            os.remove(json_path+stock_id+'.json')
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)

        try:
            Date=[]
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))

            Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            df=pd.DataFrame(list(zip(Date,Low,Open,Volume,High,Close,Adjusted_Close)),columns =['Date','Low','Open','Volume','High','Close','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
            return
        except:
            print(">>>  Historical data of "+stock_id+" exists but has no trading data")

In [4]:
json_path = os.getcwd()+os.sep+".."+os.sep+"Data"+os.sep+"json"+os.sep
csv_path = os.getcwd()+os.sep+".."+os.sep+"Data"+os.sep+"csv"+os.sep

In [5]:
if not os.path.isdir(json_path):
    os.makedirs(json_path)
if not os.path.isdir(csv_path):
    os.makedirs(csv_path)

In [6]:
def get_tickers_from_2017(country_name):
    ticker_file_path = "Assets"+os.sep+"Yahoo Ticker Symbols - September 2017.xlsx"
    df = pd.read_excel(ticker_file_path)
    
    df = df.drop(df.columns[[5, 6, 7]], axis=1)
    headers = df.iloc[2]
    df  = pd.DataFrame(df.values[3:], columns=headers)
    
    df = df[df["Country"].str.lower().str.contains(country_name.lower()) == True]
    print("Total stocks:",len(df))
    return df


In [7]:
country_name = "USA"
us_2017_tickers = get_tickers_from_2017(country_name)
us_2017_tickers.head()

Total stocks: 22169


2,Ticker,Name,Exchange,Category Name,Country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA


In [8]:
us_2017_tickers.to_csv( os.getcwd()+os.sep+".."+os.sep+"Data"+os.sep+country_name+'.csv', sep=',', index=None)

In [9]:
def get_tickers_from_names(company_list, df):
    ticker_list=[]
    for company in company_list:
        try:
            exact_company_name = (difflib.get_close_matches(company, df['Name'])[0])
            ticker_for_the_company = df.loc[df['Name'] == exact_company_name, 'Ticker'].iloc[0]
            ticker_list.append(ticker_for_the_company)
        except:
            print("Company name "+company+" not found.")
    return ticker_list

In [10]:
def get_query_list(ticker_list):
    query_urls=[]
    for ticker in ticker_list:
        query_urls.append("https://query1.finance.yahoo.com/v8/finance/chart/"+ticker+"?symbol="+ticker+
                          "&period1=0&period2=9999999999&interval=5d&includePrePost=true&events=div%2Csplit")
    return query_urls

In [11]:
# Potential queries

# https://query1.finance.yahoo.com/v8/finance/chart/%5EGSPC?p=^GSPC
# https://query1.finance.yahoo.com/v8/finance/chart/AAPL?symbol=AAPL&period1=0&period2=9999999999&interval=1d&includePrePost=true&events=div%2Csplit
# https://query1.finance.yahoo.com/v8/finance/chart/%5EGSPC?p=^GSPC&period1=0&period2=9999999999&interval=1wk&includePrePost=true&events=div%2Csplit

us_2017_tickers.head()

2,Ticker,Name,Exchange,Category Name,Country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA


In [12]:
queries = ['https://query1.finance.yahoo.com/v8/finance/chart/%5EGSPC?p=^GSPC&period1=0&period2=9999999999&interval=1wk&includePrePost=true&events=div%2Csplit']
desired_company_list = [ 'Apple Inc.', 'Amazon.com, Inc.', 'Alphabet Inc.' ]


In [13]:
queries += get_query_list(get_tickers_from_names(desired_company_list, us_2017_tickers))


In [14]:
print(queries)

['https://query1.finance.yahoo.com/v8/finance/chart/%5EGSPC?p=^GSPC&period1=0&period2=9999999999&interval=1wk&includePrePost=true&events=div%2Csplit', 'https://query1.finance.yahoo.com/v8/finance/chart/AAPL?symbol=AAPL&period1=0&period2=9999999999&interval=5d&includePrePost=true&events=div%2Csplit', 'https://query1.finance.yahoo.com/v8/finance/chart/AMZN?symbol=AMZN&period1=0&period2=9999999999&interval=5d&includePrePost=true&events=div%2Csplit', 'https://query1.finance.yahoo.com/v8/finance/chart/GOOG?symbol=GOOG&period1=0&period2=9999999999&interval=5d&includePrePost=true&events=div%2Csplit']


In [ ]:
def get_historic_price(query_url,json_path,csv_path):
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)
    
    stock_id=query_url.split("&period")[0].split("symbol=")[1]
    
    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists, Updating data...")

        try:
            with urllib.request.urlopen(query_url) as url:
                parsed = json.loads(url.read().decode())
        except:
            print("|||  Historical data of "+stock_id+" doesn't exist")
            return
    
    else:
        if os.path.exists(json_path+stock_id+'.json'):
            os.remove(json_path+stock_id+'.json')
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)

        try:
            Date=[]
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))

            Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            df=pd.DataFrame(list(zip(Date,Low,Open,Volume,High,Close,Adjusted_Close)),columns =['Date','Low','Open','Volume','High','Close','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
            return
        except:
            print(">>>  Historical data of "+stock_id+" exists but has no trading data")

In [15]:
with Pool(processes=len(queries)) as pool:
    pool.starmap(get_historic_price, zip(queries, itertools.repeat(json_path), itertools.repeat(csv_path)))
print("All downloads completed !") 

True
True
True
True


IndexError: list index out of range